In [4]:
import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [5]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
df.drop(columns=['day'])
# display the table


,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
29/02/20,105
01/03/20,140
02/03/20,166
03/03/20,229
04/03/20,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [6]:
fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases')
fig.show()

[0.65      0.4807113 1.       ]
`ftol` termination condition is satisfied.
Function evaluations 16, initial cost 1.2133e+01, final cost 8.4956e-03, first-order optimality 1.48e-07.
            Exponential model
            x0 = -69.17899055049197
            y0 = -1089.8330600455158
            k = 0.05486831582377732
            
`gtol` termination condition is satisfied.
Function evaluations 20, initial cost 3.2396e+00, final cost 1.9987e-04, first-order optimality 2.01e-10.
        Sigmoid model
        x0 = 77.6955068615055
        y0 = -91.16544438034974
        c = 4603.420635530691
        k = 0.17300629947463567
        asymptot = 4512.255191150341
        flex = 77.6955068615055, 2210.5448733849958
        
`ftol` termination condition is satisfied.
Function evaluations 21, initial cost 1.3268e+00, final cost 3.5096e-04, first-order optimality 6.53e-08.
        Sigmoid derivative model
        x0 = 87.95205744414665
        y0 = -172.0806191858915
        c = 108801.3226776617